#### Week 6: File ingestion and schema validation
Due No Due Date Points 100 Submitting a website url

Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size

In [2]:
import pandas as pd
import datetime

In [3]:
import os
 
file_size = os.stat('Income_Expense_Data_Big.csv')
print("Size of file :", file_size.st_size/(1024*1024*1024), "gb")

Size of file : 0.11675313860177994 gb


Reading file using Pandas

In [4]:
now = datetime.datetime.now()
print ("Current date and time before: ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))
df = pd.read_csv("Income_Expense_Data_Big.csv")
now = datetime.datetime.now()
print ("Current date and time after: ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time before: 
2022-09-13 12:15:57
Current date and time after: 
2022-09-13 12:15:59


Reding file using DASK

In [5]:
import dask.dataframe as dd
now = datetime.datetime.now()
print ("Current date and time before: ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))
dask_df = dd.read_csv("Income_Expense_Data_Big.csv")
now = datetime.datetime.now()
print ("Current date and time after: ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time before: 
2022-09-13 12:15:59
Current date and time after: 
2022-09-13 12:15:59


As we can see that DASK is much more faster than Pandas

In [6]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    print("dfcols:",df.columns)
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    print("expected_col:",expected_col)
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [7]:
%%writefile income.yaml
file_type: csv
dataset_name: testfile
file_name: Income_Expense_Data_Big
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Age
    - Income
    - Expense

Overwriting income.yaml


In [8]:
# Read config file
import testutility as util
config_data = util.read_config_file("income.yaml")

In [9]:
config_data['inbound_delimiter']

','

In [10]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'Income_Expense_Data_Big',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Age', 'Income', 'Expense']}

In [11]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("Income_Expense_Data_Big.csv",delimiter=',')
df_sample.head()

,Age,Income,Expense,Age___1,Income__1,Expense1,Age2,Income2___,Expense2,Age3,...,Expense4,Age5,Income5,Expense5,Age6,Income6,Expense6,Age7,Income7,Expense7
0,25,40000,20000,25,40000,20000,25,40000,20000,25,...,20000,25,40000,20000,25,40000,20000,25,40000,20000
1,26,35000,18000,26,35000,18000,26,35000,18000,26,...,18000,26,35000,18000,26,35000,18000,26,35000,18000
2,27,90000,60000,27,90000,60000,27,90000,60000,27,...,60000,27,90000,60000,27,90000,60000,27,90000,60000
3,32,70000,28000,32,70000,28000,32,70000,28000,32,...,28000,32,70000,28000,32,70000,28000,32,70000,28000
4,31,75000,32000,31,75000,32000,31,75000,32000,31,...,32000,31,75000,32000,31,75000,32000,31,75000,32000


In [12]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

C:\Users\priya\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Age,Income,Expense,Age___1,Income__1,Expense1,Age2,Income2___,Expense2,Age3,...,Expense4,Age5,Income5,Expense5,Age6,Income6,Expense6,Age7,Income7,Expense7
0,25,40000,20000,25,40000,20000,25,40000,20000,25,...,20000,25,40000,20000,25,40000,20000,25,40000,20000
1,26,35000,18000,26,35000,18000,26,35000,18000,26,...,18000,26,35000,18000,26,35000,18000,26,35000,18000
2,27,90000,60000,27,90000,60000,27,90000,60000,27,...,60000,27,90000,60000,27,90000,60000,27,90000,60000
3,32,70000,28000,32,70000,28000,32,70000,28000,32,...,28000,32,70000,28000,32,70000,28000,32,70000,28000
4,31,75000,32000,31,75000,32000,31,75000,32000,31,...,32000,31,75000,32000,31,75000,32000,31,75000,32000


In [13]:
#validate the header of the file
util.col_header_val(df,config_data)

dfcols: Index(['age', 'income', 'expense', 'age___1', 'income__1', 'expense1    ',
       'age2', 'income2___', 'expense2', 'age3', 'income3', 'expense3', 'age4',
       'income4', 'expense4', 'age5', 'income5', 'expense5', 'age6', 'income6',
       'expense6', 'age7', 'income7', 'expense7'],
      dtype='object')
expected_col: ['age', 'expense', 'income']
column name and column length validation failed
Following File columns are not in the YAML file ['expense1', 'age5', 'income_1', 'age_1', 'age7', 'age2', 'income6', 'income5', 'expense6', 'expense5', 'expense3', 'expense4', 'age6', 'income4', 'age4', 'income2', 'expense2', 'income7', 'age3', 'expense7', 'income3']
Following YAML columns are not in the file uploaded []


0

In [14]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['age', 'income', 'expense', 'age_1', 'income_1', 'expense1', 'age2',
       'income2', 'expense2', 'age3', 'income3', 'expense3', 'age4', 'income4',
       'expense4', 'age5', 'income5', 'expense5', 'age6', 'income6',
       'expense6', 'age7', 'income7', 'expense7'],
      dtype='object')
columns of YAML are: ['Age', 'Income', 'Expense']


In [15]:
df = pd.read_csv("./Income_Expense_Data_Big.csv")
df

,Age,Income,Expense,Age___1,Income__1,Expense1,Age2,Income2___,Expense2,Age3,...,Expense4,Age5,Income5,Expense5,Age6,Income6,Expense6,Age7,Income7,Expense7
0,25,40000,20000,25,40000,20000,25,40000,20000,25,...,20000,25,40000,20000,25,40000,20000,25,40000,20000
1,26,35000,18000,26,35000,18000,26,35000,18000,26,...,18000,26,35000,18000,26,35000,18000,26,35000,18000
2,27,90000,60000,27,90000,60000,27,90000,60000,27,...,60000,27,90000,60000,27,90000,60000,27,90000,60000
3,32,70000,28000,32,70000,28000,32,70000,28000,32,...,28000,32,70000,28000,32,70000,28000,32,70000,28000
4,31,75000,32000,31,75000,32000,31,75000,32000,31,...,32000,31,75000,32000,31,75000,32000,31,75000,32000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036049,62,43000,11000,62,43000,11000,62,43000,11000,62,...,11000,62,43000,11000,62,43000,11000,62,43000,11000
1036050,62,43000,11000,62,43000,11000,62,43000,11000,62,...,11000,62,43000,11000,62,43000,11000,62,43000,11000
1036051,62,43000,11000,62,43000,11000,62,43000,11000,62,...,11000,62,43000,11000,62,43000,11000,62,43000,11000
1036052,62,43000,11000,62,43000,11000,62,43000,11000,62,...,11000,62,43000,11000,62,43000,11000,62,43000,11000


In [30]:
print("Total number of rows",len(df))
print("total number of columns",len(df.columns))
print("Size of file :", file_size.st_size/(1024*1024*1024), "gb")

Total number of rows 1036054
total number of columns 24
Size of file : 0.11675313860177994 gb


In [31]:
df.to_csv(r'data_in_text.txt', index=None, sep=config_data['outbound_delimiter'], mode='a')

In [32]:
config_data['outbound_delimiter']

'|'

In [33]:
df_txt = pd.read_csv("data_in_text.txt")
df_txt.head()

,Age|Income|Expense|Age___1|Income__1|Expense1 |Age2|Income2___|Expense2|Age3|Income3|Expense3|Age4|Income4|Expense4|Age5|Income5|Expense5|Age6|Income6|Expense6|Age7|Income7|Expense7
0,25|40000|20000|25|40000|20000|25|40000|20000|2...
1,26|35000|18000|26|35000|18000|26|35000|18000|2...
2,27|90000|60000|27|90000|60000|27|90000|60000|2...
3,32|70000|28000|32|70000|28000|32|70000|28000|3...
4,31|75000|32000|31|75000|32000|31|75000|32000|3...


Create a summary of the file:

Total number of rows,

total number of columns

file size

In [34]:
print("Total number of rows",len(df_txt))
print("total number of columns",len(df_txt.columns))
 
file_size_txt = os.stat('data_in_text.txt')
print("Size of file :", file_size_txt.st_size/(1024*1024*1024), "gb")

Total number of rows 2072109
total number of columns 1
Size of file : 0.23350627720355988 gb
